In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
import pickle
import math
from sklearn.utils import shuffle

In [2]:
#LinearRegression.py

code = """

import numpy as np

class LinearRegression:
    def __init__(self, batch_size=32, regularization=0, max_epochs=100, patience=3):
        self.batch_size = batch_size
        self.regularization = regularization
        self.max_epochs = max_epochs
        self.patience = patience
        self.weights = None
        self.bias = None
        self.learning_rate = 0.01
        self.losses = []

    def _initialize_weights(self,input_dim,output_dim):
        self.weights=np.random.randn(input_dim,output_dim)
        self.bias=np.zeros((1,output_dim))

    def _prepare_data(self,X,y):
        num_samples=X.shape[0]
        split_point=int(0.9*num_samples)
        return X[:split_point],X[split_point:],y[:split_point],y[split_point:]

    def _create_batches(self, X, y):
        num_samples=X.shape[0]
        indices=np.arange(num_samples)
        np.random.shuffle(indices)
        X,y=X[indices],y[indices]

        for start in range(0,num_samples,self.batch_size):
            end=start+self.batch_size
            yield X[start:end],y[start:end]

    def _calculate_loss(self,y_pred,y_true):
        return np.mean((y_pred-y_true)**2)

    def _optimize(self, X_batch, y_batch):
        predictions=np.dot(X_batch,self.weights)+self.bias
        loss=self._calculate_loss(predictions,y_batch)

        error= predictions-y_batch
        gradient_weights=(2/X_batch.shape[0])*np.dot(X_batch.T,error)
        gradient_bias=2*np.mean(error,axis=0)

        self.weights=self.weights-self.learning_rate*(gradient_weights+2*self.regularization*self.weights)
        self.bias=self.bias-self.learning_rate*gradient_bias

        return loss

    def fit(self, X, y, batch_size=32, regularization=0, max_epochs=100, patience=3):
        num_features=X.shape[1]
        num_outputs=y.shape[1] if y.ndim>1 else 1

        self._initialize_weights(num_features,num_outputs)
        X_train,X_val,y_train,y_val=self._prepare_data(X,y)
        y_train=y_train.reshape(-1, 1)

        best_validation_loss=float('inf')
        wait_counter=0

        for epoch in range(self.max_epochs):
            for X_batch,y_batch in self._create_batches(X_train,y_train):
                loss=self._optimize(X_batch,y_batch)

            val_predictions=np.dot(X_val,self.weights)+self.bias
            validation_loss=self._calculate_loss(val_predictions,y_val)
            self.losses.append(loss)

            if validation_loss<best_validation_loss:
                best_validation_loss=validation_loss
                best_weights=self.weights.copy()
                best_bias=self.bias.copy()
                wait_counter=0
            else:
                wait_counter+=1
                if wait_counter>=self.patience:
                    self.weights=best_weights
                    self.bias=best_bias
                    break

    def predict(self,X):
        return np.dot(X,self.weights)+self.bias

    def score(self,X,y):
        predictions=self.predict(X)
        return self._calculate_loss(predictions,y)

    def save(self, filepath):
        with open(filepath, 'wb') as f:
          pickle.dump({'weights': self.weights, 'bias': self.bias}, f)

    def load(self, filepath):
      with open(filepath, 'rb') as f:
        data = pickle.load(f)
        self.weights = data['weights']
        self.bias = data['bias']

"""

# Save to file
with open("LinearRegression.py", "w") as file:
    file.write(code)


In [ ]:
#train_regression1.py

from LinearRegression import LinearRegression
import numpy as np

iris=datasets.load_iris()
X=iris.data
y=iris.target

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=1)

iris=datasets.load_iris()
X=iris.data[:, :2]
y=iris.data[:, 3]

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=1)

lr=LinearRegression()
lr.fit(X_train,y_train)
pred1 = lr.predict(X_test)
print(pred1)
print(y_test)
print(np.concatenate((pred1.reshape(len(pred1),1), y_test.reshape(len(y_test),1)),1))

with open('model1.pkl', 'wb') as file:
    pickle.dump(lr, file)

weights,bias=lr.weights,lr.bias

print("Weights are :\n ", weights)
print("Bias are :\n", bias)

plt.plot(lr.losses)
plt.title("Model 1 loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.savefig("Model1loss.png")
plt.show()

In [ ]:
#train_regression2.py

from LinearRegression import LinearRegression
import numpy as np

iris=datasets.load_iris()
X=iris.data
y=iris.target

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=1)

iris = datasets.load_iris()
X=iris.data[:,[0,2]]
y=iris.data[:, 3]

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=1)

lr2 = LinearRegression()
lr2.fit(X_train, y_train)
pred2 = lr2.predict(X_test)
print(pred2)
print(y_test)
print(np.concatenate((pred2.reshape(len(pred2),1), y_test.reshape(len(y_test),1)),1))

with open('model2.pkl', 'wb') as file:
    pickle.dump(lr2, file)

weights,bias=lr2.weights,lr2.bias

print("Weights are :\n ",weights)
print("Bias are :\n",bias)

plt.plot(lr2.losses)
plt.title("Model 2 loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.savefig("Model2loss.png")
plt.show()

In [ ]:
#train_regression3.py

from LinearRegression import LinearRegression
import numpy as np

iris=datasets.load_iris()
X=iris.data
y=iris.target

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=1)

iris = datasets.load_iris()
X=iris.data[:, 2:]
y=iris.data[:, 1]

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=1)

lr3 = LinearRegression()
lr3.fit(X_train, y_train)
pred3 = lr3.predict(X_test)
print(pred3)
print(y_test)
print(np.concatenate((pred3.reshape(len(pred3),1), y_test.reshape(len(y_test),1)),1))

with open('model3.pkl', 'wb') as file:
    pickle.dump(lr3, file)

weights,bias=lr3.weights,lr3.bias

print("Weights are :\n ",weights)
print("Bias are :\n",bias)

plt.plot(lr2.losses)
plt.title("Model 3 loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.savefig("Model3loss.png")
plt.show()

In [ ]:
#train_regression4.py

from LinearRegression import LinearRegression
import numpy as np

iris=datasets.load_iris()
X=iris.data
y=iris.target

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=1)

iris = datasets.load_iris()
X=iris.data[:,[0,3]]
y=iris.data[:, 2]

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=1)

lr4 = LinearRegression()
lr4.fit(X_train, y_train)
pred4 = lr4.predict(X_test)
print(pred4)
print(y_test)
print(np.concatenate((pred4.reshape(len(pred4),1), y_test.reshape(len(y_test),1)),1))

with open('model4.pkl', 'wb') as file:
    pickle.dump(lr4, file)

weights,bias=lr4.weights,lr4.bias

print("Weights are :\n ",weights)
print("Bias are :\n",bias)

plt.plot(lr4.losses)
plt.title("Model 4 loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.savefig("Model4loss.png")
plt.show()

In [ ]:
#eval_regression1.py

from LinearRegression import LinearRegression
lr = LinearRegression()

with open('model1.pkl', 'rb') as file:
    loaded_lr = pickle.load(file)

pred = loaded_lr.predict(X_test)
mse = loaded_lr.score(X_test, y_test)
print(f"Model1 MSE : " + str(mse))

In [ ]:
#eval_regression2.py

from LinearRegression import LinearRegression
lr2 = LinearRegression()

with open('model2.pkl', 'rb') as file:
    loaded_lr2 = pickle.load(file)

pred = loaded_lr2.predict(X_test)
mse = loaded_lr2.score(X_test, y_test)
print(f"Model2 MSE : " + str(mse))


In [ ]:
#eval_regression3.py

from LinearRegression import LinearRegression
lr3 = LinearRegression()

with open('model3.pkl', 'rb') as file:
    loaded_lr3 = pickle.load(file)

pred = loaded_lr3.predict(X_test)
mse = loaded_lr3.score(X_test, y_test)
print(f"Model3 MSE : " + str(mse))

In [ ]:
#eval_regression4.py

from LinearRegression import LinearRegression
lr4 = LinearRegression()

with open('model4.pkl', 'rb') as file:
    loaded_lr4 = pickle.load(file)

pred = loaded_lr4.predict(X_test)
mse = loaded_lr4.score(X_test, y_test)
print(f"Model4 MSE : " + str(mse))

In [ ]:
#train_regression_multiple.py

from LinearRegression import LinearRegression

iris=datasets.load_iris()
X=iris.data[:, :2]
y=iris.data[:, 2:]

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=1)

with open('multiple_X_train.pkl', 'wb') as f:
    pickle.dump(X_train, f)
with open('multiple_X_test.pkl', 'wb') as f:
    pickle.dump(X_test, f)
with open('multiple_y_train.pkl', 'wb') as f:
    pickle.dump(y_train, f)
with open('multiple_y_test.pkl', 'wb') as f:
    pickle.dump(y_test, f)

#####################################################################

with open('multiple_X_train.pkl', 'rb') as f:
    X_train = pickle.load(f)
with open('multiple_y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

lr5_multi = LinearRegression()
lr5_multi.fit(X_train, y_train)

with open('multi_lr5.pkl', 'wb') as f:
    pickle.dump(lr5_multi, f)

plt.plot(lr5_multi.losses)
plt.title("Model Multiple Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.savefig("ModelMultiple.jpg")
plt.show()

In [ ]:
#eval_regression_multiple.py

with open('multiple_X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)
with open('multiple_y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

lr5_multi = LinearRegression()
with open('multi_lr5.pkl', 'rb') as f:
    lr5_multi = pickle.load(f)

pred = loaded_lr4.predict(X_test)
mse = lr5_multi.score(X_test, y_test)
print(f"Multiple MSE : " + str(mse))

In [13]:
#LogisticRegression.py

code2 = '''

import numpy as np

class LogisticRegression:
    def __init__(self, batch_size=32, regularization=0, max_epochs=100, patience=3):
        self.batch_size = batch_size
        self.regularization = regularization
        self.max_epochs = max_epochs
        self.patience = patience
        self.weights = None
        self.bias = None
        self.learning_rate = 0.01
        self.losses = []

    def _initialize_weights(self, input_dim):
        self.weights=np.random.randn(input_dim,1)
        self.bias=0

    def _prepare_data(self,X,y):
        num_samples=X.shape[0]
        split_point=int(0.9*num_samples)
        return X[:split_point],X[split_point:],y[:split_point],y[split_point:]

    def _create_batches(self,X,y):
        num_samples=X.shape[0]
        indices=np.arange(num_samples)
        np.random.shuffle(indices)
        X,y=X[indices],y[indices]

        for start in range(0,num_samples,self.batch_size):
            end=start+self.batch_size
            yield X[start:end],y[start:end]

    def _sigmoid(self,z):
        return 1/(1+np.exp(-z))

    def _calculate_loss(self,y_pred,y_true):
        m=y_true.shape[0]
        epsilon=1e-10
        y_pred=np.clip(y_pred,epsilon,1-epsilon)
        loss=-(1/m)*np.sum(y_true*np.log(y_pred)+(1-y_true)*np.log(1-y_pred))
        return loss

    def _optimize(self,X_batch,y_batch):
        predictions=self._sigmoid(np.dot(X_batch,self.weights)+self.bias)
        loss=self._calculate_loss(predictions,y_batch)

        error=predictions-y_batch
        gradient_weights=(1/X_batch.shape[0])*np.dot(X_batch.T,error)
        gradient_bias=np.mean(error)

        self.weights=self.weights-self.learning_rate*(gradient_weights+2*self.regularization*self.weights)
        self.bias=self.bias-self.learning_rate*gradient_bias

        return loss

    def fit(self,X,y):
        num_features=X.shape[1]

        self._initialize_weights(num_features)
        X_train,X_val,y_train,y_val=self._prepare_data(X,y)
        y_train=y_train.reshape(-1,1)
        y_val=y_val.reshape(-1,1)

        best_validation_loss=float('inf')
        wait_counter=0

        for epoch in range(self.max_epochs):
            for X_batch,y_batch in self._create_batches(X_train,y_train):
                loss=self._optimize(X_batch,y_batch)

            val_predictions=self._sigmoid(np.dot(X_val,self.weights)+self.bias)
            validation_loss=self._calculate_loss(val_predictions,y_val)
            self.losses.append(loss)

            if validation_loss<best_validation_loss:
                best_validation_loss=validation_loss
                best_weights=self.weights.copy()
                best_bias=self.bias
                wait_counter=0
            else:
                wait_counter+=1
                if wait_counter>=self.patience:
                    self.weights=best_weights
                    self.bias=best_bias
                    break

    def predict(self,X):
        return self._sigmoid(np.dot(X,self.weights)+self.bias)

    def score(self,X,y):
        predictions=self.predict(X)
        return self._calculate_loss(predictions,y)

    def save(self,filepath):
      with open(filepath,'wb') as f:
        pickle.dump({'weights':self.weights,'bias':self.bias},f)

    def load(self,filepath):
      with open(filepath,'rb') as f:
        data=pickle.load(f)
        self.weights=data['weights']
        self.bias=data['bias']

'''
with open("LogisticRegression.py", "w") as file:
    file.write(code2)

In [ ]:
#train_classifier1.py

import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pickle

iris=datasets.load_iris()

#1.petal length/width
X=iris.data[:, 2:]
y=iris.target
X_train_p,X_test_p,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=1)


scaler_p = StandardScaler()
X_train_scaled = scaler_p.fit_transform(X_train_p)
X_test_scaled = scaler_p.transform(X_test_p)

# Save the standardized datasets using pickle
with open('processed_iris_datap.pkl', 'wb') as f:
    pickle.dump({
        'X_train': X_train_scaled,
        'y_train': y_train,
        'X_test': X_test_scaled,
        'y_test': y_test
    }, f)


from LogisticRegression import LogisticRegression
from mlxtend.plotting import plot_decision_regions

with open('processed_iris_datap.pkl', 'rb') as f:
    data = pickle.load(f)

X_train_scaled = data['X_train']
y_train = data['y_train']
X_test_scaled = data['X_test']
y_test = data['y_test']

X_train = data['X_train']
y_train = data['y_train']

lgr = LogisticRegression()
lgr.fit(X_train, y_train)

with open('modelp.pkl', 'wb') as f:
    pickle.dump(lgr, f)

plot_decision_regions(X_train, y_train, clf=lgr)
plt.title("Logistic Regression")
plt.xlabel("Petal Length")
plt.ylabel("Petal Width")
plt.savefig("LogisticPetalLengthWidth.jpg")
plt.show()

In [ ]:
#eval_classifier1.py

with open('processed_iris_datap.pkl', 'rb') as f:
    data = pickle.load(f)

X_train_scaled = data['X_train']
y_train = data['y_train']
X_test_scaled = data['X_test']
y_test = data['y_test']

X_test = data['X_test']
y_test = data['y_test']

with open('modelp.pkl', 'rb') as f:
    lgr = pickle.load(f)

modelp_lgr = lgr.predict(X_test)
modelp_lgr = np.argmax(modelp_lgr, axis=1) #get the class with highest probability
acc = accuracy_score(y_test, modelp_lgr)

print(f'Accuracy for Petal Length/Width: {acc:.4f}')

In [ ]:
#train_classifier2.py

from sklearn.preprocessing import StandardScaler

iris=datasets.load_iris()

#2.sepal length/width
X=iris.data[:, :2]
y=iris.target
X_train_s,X_test_s,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=1)

scaler_p = StandardScaler()
X_train_scaled = scaler_p.fit_transform(X_train_s)
X_test_scaled = scaler_p.transform(X_test_s)

import pickle

with open('processed_iris_datas.pkl', 'wb') as f:
    pickle.dump({
        'X_train': X_train_scaled,
        'y_train': y_train,
        'X_test': X_test_scaled,
        'y_test': y_test
    }, f)


from LogisticRegression import LogisticRegression
from mlxtend.plotting import plot_decision_regions

with open('processed_iris_datas.pkl', 'rb') as f:
    data = pickle.load(f)

X_train_scaled = data['X_train']
y_train = data['y_train']
X_test_scaled = data['X_test']
y_test = data['y_test']


X_train = data['X_train']
y_train = data['y_train']

lgr = LogisticRegression()
lgr.fit(X_train, y_train)

with open('models.pkl', 'wb') as f:
    pickle.dump(lgr, f)

plot_decision_regions(X_train, y_train,clf=lgr)
plt.title("Logistic Regression")
plt.xlabel("Sepal Length")
plt.ylabel("Sepal Width")
plt.savefig("LogisticSepalLengthWidth.jpg")
plt.show()

In [ ]:
#eval_classifier2.py

with open('processed_iris_datas.pkl', 'rb') as f:
    data = pickle.load(f)

X_train_scaled = data['X_train']
y_train = data['y_train']
X_test_scaled = data['X_test']
y_test = data['y_test']

X_test = data['X_test']
y_test = data['y_test']

with open('models.pkl', 'rb') as f:
    lgr = pickle.load(f)

models_lgr = lgr.predict(X_test)
models_lgr = np.argmax(models_lgr, axis=1)
acc = accuracy_score(y_test, models_lgr)

print(f'Accuracy for Sepal Length/Width: {acc:.4f}')

In [18]:
#train_classifier3.py

import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle

#3.all features
iris=datasets.load_iris()
X=iris.data
y=iris.target
X_train_all,X_test_all,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=1)

scaler_p = StandardScaler()
X_train_scaled = scaler_p.fit_transform(X_train_all)
X_test_scaled = scaler_p.transform(X_test_all)

with open('processed_iris_data_all.pkl', 'wb') as f:
    pickle.dump({
        'X_train': X_train_scaled,
        'y_train': y_train,
        'X_test': X_test_scaled,
        'y_test': y_test
    }, f)


from LogisticRegression import LogisticRegression

with open('processed_iris_data_all.pkl', 'rb') as f:
    data = pickle.load(f)

X_train_scaled = data['X_train']
y_train = data['y_train']
X_test_scaled = data['X_test']
y_test = data['y_test']


X_train = data['X_train']
y_train = data['y_train']

lgr = LogisticRegression()
lgr.fit(X_train, y_train)

with open('model_all.pkl', 'wb') as f:
    pickle.dump(lgr, f)

In [ ]:
#eval_classifier3.py

with open('processed_iris_data_all.pkl', 'rb') as f:
    data = pickle.load(f)

X_train_scaled = data['X_train']
y_train = data['y_train']
X_test_scaled = data['X_test']
y_test = data['y_test']

X_test = data['X_test']
y_test = data['y_test']

with open('model_all.pkl', 'rb') as f:
    lgr = pickle.load(f)

model_all_lgr = lgr.predict(X_test)
model_all_lgr = np.argmax(model_all_lgr, axis=1)
acc = accuracy_score(y_test, model_all_lgr)

print(f'Accuracy for All features: {acc:.4f}')